참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

(`sympy` 1.0 이후에는 radian -> degree 변환 기능이 `mpmath` 로 독립)<br>
(For `sympy` 1.0 or later, `mpmath` is an external dependency)

In [ ]:
try:
    from sympy.mpmath import degrees as sy_degrees
except ImportError: 
    import sys
    from mpmath import degrees as sy_degrees

## 예제 03.006

### 문제에서 주어진 변수<br>Given Parameters

다음과 같은 변수들이 문제에서 주어졌다.

In [ ]:
L_m, t_m, G_st_Pa, w_1_m, w_2_m, h_m = sy.symbols('L[m], t[m], G_{st}[Pa], w_{1}[m], w_{2}[m], h[m]')

mm 단위 변수도 준비해 둔다.

In [ ]:
t_mm = 1e-3 * t_m
w_1_mm = 1e-3 * w_1_m
w_2_mm = 1e-3 * w_2_m
h_mm = 1e-3 * h_m

변수값을 나중에 사용하기 위해 `dict` 형태로 저장해 둔다.

In [ ]:
subs_dict = {L_m: 3,  # 각관의 길이
             t_mm: 11.25,  # 각관의 벽 두께
             G_st_Pa: 80e9,  # 철의 전단탄성계수
             w_1_mm: 180,  # 각관의 폭 1
             w_2_mm: 120,  # 각관의 폭 2
             h_mm: 150,  # 각관의 높이
            }
subs_dict[t_m] = subs_dict[t_mm] * 1e-3  # 각관의 벽 두께
subs_dict[w_1_m] = subs_dict[w_1_mm] * 1e-3  # 각관의 폭 1
subs_dict[w_2_m] = subs_dict[w_2_mm] * 1e-3  # 각관의 폭 2
subs_dict[h_m] = subs_dict[h_mm] * 1e-3  # 각관의 높이
subs_dict

### 1) 튜브의 비틀림 강성

축방향 하중의 경우 강성계수 $k$는 다음과 같이 구할 수 있다.
$$F=k\delta\\
k=\frac{F}{\delta} \\
\delta = \frac{FL}{EA}\\
k=\frac{EA}{L}
$$

비틀림 강성도 다음과 같이 구할 수 있을 것이다.
$$T=k\theta\\
k=\frac{T}{\theta}\\
\theta=\frac{TL}{G}\frac{S}{4A_0^2t}\\
$$

$$k=\frac{G}{L}\frac{4A_0^2t}{S}
$$

두께의 중심선 안 면적은 다음과 같이 사다리꼴의 넓이로 구할 수 있다.

In [ ]:
A0_m2 = (w_1_m + w_2_m) * h_m /2
A0_m2

변수값을 대입하면 다음과 같다.

In [ ]:
A0_m2.subs(subs_dict)

단위를 포함하여 표시하자면 다음과 같다.

In [ ]:
disp.display(disp.Math('$A_0 = %g(m^2)$' % (A0_m2.subs(subs_dict))))
disp.display(disp.Math('$A_0 = %g(mm^2)$' % (1e6*A0_m2.subs(subs_dict))))

두께가 $t$ 일 때 $\frac{t}{2}$ 지점을 이은 중심선의 길이는 다음과 같다.

In [ ]:
S_m = w_1_m + w_2_m + 2*sy.sqrt(((w_1_m - w_2_m)/2)**2 + h_m **2)
S_m

단면이 사다리꼴이므로 긴 변과 짧은 변의 길이를 더하고 $(w_1 + w_2)$ 빗변의 길이에 2를 곱하여 더하였다. 한 빗변의 길이는 피타고라스 법칙을 이용하여 구하였다. $\left(\sqrt{h^2+(\frac{w_1-w_2}{2})^2}\right)$

변수값을 대입하면 다음과 같다.

In [ ]:
S_m.subs(subs_dict)

단위를 포함하여 표시하자면 다음과 같다.

In [ ]:
disp.display(disp.Math('$S = %g(m)$' % (S_m.subs(subs_dict))))
disp.display(disp.Math('$S = %g(mm)$' % (1e3*S_m.subs(subs_dict))))

비틀림각 변위는 다음과 같다.
$$\theta[rad] = \frac{T[Nm] L[m] S[m]}{4 G[Nm^{-2}] [A_0[m^2]]^2 t[m]}$$

In [ ]:
T_Nm = sy.symbols('T[Nm]')  # 아래 식에 쓰기 위해 지정
theta_rad = T_Nm * L_m * S_m / (4 * G_st_Pa * A0_m2**2 * t_m)
theta_rad

비틀림 강성은 다음과 같이 구할 수 있다.

In [ ]:
k_Nm_rad = T_Nm / theta_rad
k_Nm_rad

In [ ]:
k_Nm_rad.subs(subs_dict)

각도의 단위를 radian 에서 도 로 바꾸면 다음과 같이 된다.

In [ ]:
theta_deg = sy_degrees(theta_rad)
k_Nm_deg = T_Nm / theta_deg
k_Nm_deg

In [ ]:
k_Nm_deg.subs(subs_dict)

### 2) 0.5도 회전시 벽에서의 응력

0.5 도 회전을 위해 필요한 토크는 다음과 같다.
$$T=k \theta$$

In [ ]:
T_0_5_deg_Nm = k_Nm_deg * 0.5
T_0_5_deg_Nm.subs(subs_dict)

전단류는 다음과 같다.
$$q = \frac{T}{2A_0}$$

In [ ]:
q_0_5_deg_N_m = T_0_5_deg_Nm / (2 * A0_m2)
q_0_5_deg_N_m.subs(subs_dict)

전단응력은 다음과 같다.
$$\tau=\frac{q}{t}$$

In [ ]:
tau_0_5_deg_Pa = q_0_5_deg_N_m / t_m
tau_Pa = tau_0_5_deg_Pa.subs(subs_dict)
tau_Pa

MPa 단위로 바꾸어 표시하면 다음과 같다.

In [ ]:
disp.display(disp.Math('$\\tau = %g(MPa)$' % (tau_Pa*1e-6)))